# 라이브러리 import 및 초기설정

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
import time

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 화면크기 설정
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

# 데이터로딩

In [ ]:
# application_train 데이터 로딩
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print("Training Data Shape: ", app_train.shape)
app_train.head()

In [ ]:
# application_test 데이터 로딩
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print("Testing Data Shape: ", app_test.shape)
app_test.head()

# application_train 데이터에 대한 EDA

In [ ]:
app_train.info()

In [ ]:
app_train.isnull().sum()

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].value_counts() / app_train.shape[0]

In [ ]:
app_train['TARGET'].astype(int).plot.hist()

In [ ]:
app_train['AMT_INCOME_TOTAL'].hist() # 소득데이터 확인

In [ ]:
sns.distplot(app_train['AMT_INCOME_TOTAL'])

In [ ]:
sns.boxplot(app_train['AMT_INCOME_TOTAL'])

In [ ]:
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]

In [ ]:
app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]['AMT_INCOME_TOTAL'].hist()

In [ ]:
sns.distplot(app_train[app_train['AMT_INCOME_TOTAL'] < 1000000]['AMT_INCOME_TOTAL'])

In [ ]:
cond0 = (app_train['TARGET'] == 0)
cond1 = (app_train['TARGET'] == 1)
cond_amt = (app_train['AMT_INCOME_TOTAL'] < 500000)

sns.distplot(app_train[cond0 & cond_amt]['AMT_INCOME_TOTAL'], label='0', color='blue')
sns.distplot(app_train[cond1 & cond_amt]['AMT_INCOME_TOTAL'], label='1', color='red')

In [ ]:
sns.violinplot(x='TARGET', y='AMT_INCOME_TOTAL', data=app_train[cond_amt])

In [ ]:
def show_hist_by_target(df, columns):
    cond_0 = (df['TARGET'] == 0)
    cond_1 = (df['TARGET'] == 1)
    
    for column in columns:
        fig, ax = plt.subplots(figsize=(12, 4), nrows=1, ncols=2, squeeze=False)
        sns.violinplot(x='TARGET', y=column, data=df, ax=ax[0][0])
        sns.distplot(df[cond_0][column], label='0', color='blue', ax=ax[0][1])
        sns.distplot(df[cond_1][column], label='1', color='red', ax=ax[0][1])

In [ ]:
columns = ['AMT_INCOME_TOTAL','AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
           'DAYS_REGISTRATION', 'DAYS_LAST_PHONE_CHANGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'EXT_SOURCE_1', 
           'EXT_SOURCE_2', 'EXT_SOURCE_3', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
           'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

show_hist_by_target(app_train, columns)

In [ ]:
# 그래프를 통해 각각의 값을 살펴본 결과, 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3'은
# 다른 속성보다 0일 때와 1일 때의 구분이 명확하므로 주목할 필요가 있음

In [ ]:
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

In [ ]:
app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].isnull().sum()

In [ ]:
app_train['EXT_SOURCE_3'].value_counts(dropna=False)

In [ ]:
# 서로간의 상관 관계
ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

In [ ]:
# 각 EXT_SOURCE 를 TARGET값 별로 나눠서 분포를 살핌
plt.figure(figsize=(10,12))
plt.subplot(3,1,1);
sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'EXT_SOURCE_1'],label='target==0', color='BLUE')
sns.kdeplot(app_train.loc[app_train['TARGET']==1, 'EXT_SOURCE_1'],label='target==1', color='RED');

In [ ]:
plt.figure(figsize=(10,12))

for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0', color='BLUE')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1', color='RED')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
    plt.tight_layout(h_pad=2.5)

In [ ]:
object_columns = app_train.dtypes[app_train.dtypes == 'object'].index.tolist()
object_columns

In [ ]:
def show_category_by_target(df, columns):
    for column in columns:
        chart = sns.catplot(x=column, col='TARGET', data=df, kind='count')
        chart.set_xticklabels(rotation=65)

show_category_by_target(app_train, object_columns)

In [ ]:
print(app_train['CODE_GENDER'].value_counts()/app_train.shape[0])
print('\n연체인 경우\n',app_train[cond1]['CODE_GENDER'].value_counts()/app_train[cond1].shape[0])
print('\n연체가 아닌 경우\n',app_train[cond0]['CODE_GENDER'].value_counts()/app_train[cond0].shape[0])

In [ ]:
# 데이터 셋 결합
apps = pd.concat([app_train, app_test])
print(apps.shape)

# previous_application 데이터에 대한 EDA

In [ ]:
# previous_application 데이터 로딩
prev_app = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
print(prev_app.shape, apps.shape)
prev_app.head()

In [ ]:
# merge를 이용하여 조인
prev_app_outer = prev_app.merge(apps['SK_ID_CURR'], on='SK_ID_CURR', how='outer', indicator=True)
prev_app_outer.head()

In [ ]:
prev_app_outer['_merge'].value_counts()

In [ ]:
# Null 값을 확인하고 전체 데이터에서 비율을 보여주는 함수
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
prev_app.info()

In [ ]:
missing_data(prev_app).head(20)

In [ ]:
# SK_ID_CURR 당 평균 SK_ID_PREV 건수 확인
prev_app.groupby('SK_ID_CURR').count()

In [ ]:
prev_app.groupby('SK_ID_CURR')['SK_ID_CURR'].count()

In [ ]:
sns.boxplot(prev_app.groupby('SK_ID_CURR')['SK_ID_CURR'].count())

In [ ]:
app_prev_target = prev_app.merge(app_train[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
app_prev_target.shape

In [ ]:
num_columns = app_prev_target.dtypes[app_prev_target.dtypes != 'object'].index.tolist()
num_columns

In [ ]:
num_columns = [column for column in num_columns if column not in ['SK_ID_PREV', 'SK_ID_CURR', 'TARGET']]
num_columns

In [ ]:
show_hist_by_target(app_prev_target, num_columns)

In [ ]:
print(app_prev_target.groupby('TARGET').agg({'AMT_ANNUITY': ['mean', 'median', 'count']}))
print(app_prev_target.groupby('TARGET').agg({'AMT_APPLICATION': ['mean', 'median', 'count']}))
print(app_prev_target.groupby('TARGET').agg({'AMT_CREDIT': ['mean', 'median', 'count']}))

In [ ]:
object_columns2 = app_prev_target.dtypes[app_prev_target.dtypes=='object'].index.tolist()
object_columns2

In [ ]:
show_category_by_target(app_prev_target, object_columns2)

In [ ]:
# NAME_CONTRACT_TYPE은 TARGET=1일때 CASH_LOAN의 비중이 약간 높음
# NAME_CONTRACT_STATUS(대출허가상태)는 TARGET=1일때 상대적으로 TARGET=0 대비 Refused의 비율이 높음
# NAME_PAYMENT_TYPE(대출납부방법)는 TARGET=1일때 상대적으로 TARGET=0 대비 XNA의 비율이 약간 높음

In [ ]:
pd.crosstab(app_prev_target.TARGET, app_prev_target.NAME_CONTRACT_TYPE, dropna=False, normalize='all')

In [ ]:
pd.crosstab(app_prev_target.TARGET, app_prev_target.NAME_CONTRACT_STATUS, dropna=False, normalize='all')

In [ ]:
pd.crosstab(app_prev_target.NAME_CONTRACT_TYPE, app_prev_target.NAME_CONTRACT_STATUS, dropna=False)

In [ ]:
app_prev_target.groupby(['NAME_CONTRACT_TYPE', 'NAME_GOODS_CATEGORY']).agg({'AMT_ANNUITY': ['mean', 'median', 'count', 'max']})

In [ ]:
app_train.groupby(['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE']).agg({'AMT_INCOME_TOTAL': ['mean', 'median', 'count', 'max']})

# Feature Engineering

In [ ]:
prev_group = prev_app.groupby('SK_ID_CURR')
prev_group.head()

In [ ]:
prev_agg = pd.DataFrame()
prev_agg['CNT'] = prev_group['SK_ID_CURR'].count()
prev_agg.head()

In [ ]:
prev_agg['AVG_CREDIT'] = prev_group['AMT_CREDIT'].mean()
prev_agg['MAX_CREDIT'] = prev_group['AMT_CREDIT'].max()
prev_agg['MIN_CREDIT'] = prev_group['AMT_CREDIT'].min()
prev_agg.head()

In [ ]:
prev_agg1 = prev_group['AMT_CREDIT'].agg(['mean', 'max', 'min'])
prev_agg2 = prev_group['AMT_ANNUITY'].agg(['mean', 'max', 'min'])

prev_agg = prev_agg1.merge(prev_agg2, on='SK_ID_CURR', how='inner')
prev_agg.head()

In [ ]:
agg_dict = {
    'SK_ID_CURR':['count'],
    'AMT_CREDIT':['mean', 'max', 'sum'],
    'AMT_ANNUITY':['mean', 'max', 'sum'], 
    'AMT_APPLICATION':['mean', 'max', 'sum'],
    'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'],
    'AMT_GOODS_PRICE':['mean', 'max', 'sum']
}

prev_amt_agg = prev_group.agg(agg_dict)
prev_amt_agg.head()

In [ ]:
# MultiIndex로 되어 있는 컬럼명 확인
prev_amt_agg.columns

In [ ]:
prev_amt_agg.columns = ['PREV_' + ('_').join(column).upper() for column in prev_amt_agg.columns.ravel()]
prev_amt_agg.head()

In [ ]:
prev_app['PREV_CREDIT_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_CREDIT']
prev_app['PREV_GOODS_DIFF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_GOODS_PRICE']
prev_app['PREV_CREDIT_APPL_RATIO'] = prev_app['AMT_CREDIT']/prev_app['AMT_APPLICATION']
prev_app['PREV_ANNUITY_APPL_RATIO'] = prev_app['AMT_ANNUITY']/prev_app['AMT_APPLICATION']
prev_app['PREV_GOODS_APPL_RATIO'] = prev_app['AMT_GOODS_PRICE']/prev_app['AMT_APPLICATION']

In [ ]:
prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
prev_app['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev_app['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
# 첫번째 만기일과 마지막 만기일까지의 기간
prev_app['PREV_DAYS_LAST_DUE_DIFF'] = prev_app['DAYS_LAST_DUE_1ST_VERSION'] - prev_app['DAYS_LAST_DUE']

In [ ]:
prev_app.info()

In [ ]:
all_pay = prev_app['AMT_ANNUITY'] * prev_app['CNT_PAYMENT']
prev_app['PREV_INTERESTS_RATE'] = (all_pay/prev_app['AMT_CREDIT'] - 1)/prev_app['CNT_PAYMENT']
prev_app.iloc[:, -7:].head(10)

In [ ]:
agg_dict = {
     # 기존 컬럼
    'SK_ID_CURR':['count'],
    'AMT_CREDIT':['mean', 'max', 'sum'],
    'AMT_ANNUITY':['mean', 'max', 'sum'], 
    'AMT_APPLICATION':['mean', 'max', 'sum'],
    'AMT_DOWN_PAYMENT':['mean', 'max', 'sum'],
    'AMT_GOODS_PRICE':['mean', 'max', 'sum'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
    # 가공 컬럼
    'PREV_CREDIT_DIFF':['mean', 'max', 'sum'], 
    'PREV_CREDIT_APPL_RATIO':['mean', 'max'],
    'PREV_GOODS_DIFF':['mean', 'max', 'sum'],
    'PREV_GOODS_APPL_RATIO':['mean', 'max'],
    'PREV_DAYS_LAST_DUE_DIFF':['mean', 'max', 'sum'],
    'PREV_INTERESTS_RATE':['mean', 'max']
}
agg_dict

In [ ]:
prev_amt_agg = prev_group.agg(agg_dict)
prev_amt_agg.columns = ['PREV_'+('_').join(column).upper() for column in prev_amt_agg.columns.ravel()]
prev_amt_agg.head()

In [ ]:
prev_amt_agg.shape

In [ ]:
prev_app['NAME_CONTRACT_STATUS'].value_counts()

In [ ]:
cond_refused = (prev_app['NAME_CONTRACT_STATUS'] == 'Refused')
prev_refused = prev_app[cond_refused]
prev_refused.shape, prev_app.shape

In [ ]:
prev_refused_agg = prev_refused.groupby('SK_ID_CURR')['SK_ID_CURR'].count()
prev_refused_agg.shape, prev_amt_agg.shape

In [ ]:
prev_refused_agg

In [ ]:
prev_refused_agg.reset_index(name='PREV_REFUSED_COUNT')

In [ ]:
prev_refused_agg = prev_refused_agg.reset_index(name='PREV_REFUSED_COUNT')
prev_amt_agg = prev_amt_agg.reset_index()

prev_amt_refused_agg = prev_amt_agg.merge(prev_refused_agg, on='SK_ID_CURR', how='left')
prev_amt_refused_agg.head(10)

In [ ]:
prev_amt_refused_agg['PREV_REFUSED_COUNT'].value_counts(dropna=False)

In [ ]:
# 계산된 PREV_REFUSED_COUNT 중 Null값은 0 으로 변경하고 SK_ID_CURR 개별 건수 대비 PREV_REFUSED_COUNT 비율 계산
prev_amt_refused_agg = prev_amt_refused_agg.fillna(0)
prev_amt_refused_agg['PREV_REFUSE_RATIO'] = prev_amt_refused_agg['PREV_REFUSED_COUNT'] / prev_amt_refused_agg['PREV_SK_ID_CURR_COUNT']
prev_amt_refused_agg.head(10)

In [ ]:
prev_refused_appr_group = prev_app[prev_app['NAME_CONTRACT_STATUS'].isin(['Approved', 'Refused'])].groupby(['SK_ID_CURR', 'NAME_CONTRACT_STATUS'])
prev_refused_appr_agg = prev_refused_appr_group['SK_ID_CURR'].count().unstack()
prev_refused_appr_agg.head(10)

In [ ]:
prev_refused_appr_agg = prev_refused_appr_agg.fillna(0)
prev_refused_appr_agg.columns = ['PREV_APPROVED_COUNT', 'PREV_REFUSED_COUNT']
prev_refused_appr_agg = prev_refused_appr_agg.reset_index()
prev_refused_appr_agg.head(10)

In [ ]:
prev_amt_agg.head()

In [ ]:
prev_agg = prev_amt_agg.merge(prev_refused_appr_agg, on='SK_ID_CURR', how='left')

# SK_ID_CURR별 과거 대출건수 대비 APPROVED_COUNT 및 REFUSED_COUNT 비율 생성
prev_agg['PREV_REFUSED_RATIO'] = prev_agg['PREV_REFUSED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']
prev_agg['PREV_APPROVED_RATIO'] = prev_agg['PREV_APPROVED_COUNT']/prev_agg['PREV_SK_ID_CURR_COUNT']

In [ ]:
# 'PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT' 컬럼 drop 
prev_agg = prev_agg.drop(['PREV_REFUSED_COUNT', 'PREV_APPROVED_COUNT'], axis=1)

In [ ]:
prev_agg.head(30)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head()

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)

In [ ]:
apps['APPS_EXT_SOURCE_MEAN'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
apps['APPS_EXT_SOURCE_STD'] = apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_MEAN', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
apps['APPS_EXT_SOURCE_STD'] = apps['APPS_EXT_SOURCE_STD'].fillna(apps['APPS_EXT_SOURCE_STD'].mean())

In [ ]:
apps[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APPS_EXT_SOURCE_MEAN', 'APPS_EXT_SOURCE_STD']].head(10)

In [ ]:
# AMT_CREDIT 관련 Feature 가공
apps['APPS_ANNUITY_CREDIT_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_CREDIT']
apps['APPS_GOODS_CREDIT_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_CREDIT']
apps['APPS_CREDIT_GOODS_DIFF'] = apps['AMT_CREDIT'] - apps['AMT_GOODS_PRICE']

In [ ]:
# AMT_INCOME_TOTAL 비율로 대출 금액 관련 피처 가공
apps['APPS_ANNUITY_INCOME_RATIO'] = apps['AMT_ANNUITY']/apps['AMT_INCOME_TOTAL']
apps['APPS_CREDIT_INCOME_RATIO'] = apps['AMT_CREDIT']/apps['AMT_INCOME_TOTAL']
apps['APPS_GOODS_INCOME_RATIO'] = apps['AMT_GOODS_PRICE']/apps['AMT_INCOME_TOTAL']

In [ ]:
# 가족수를 고려한 가처분 소득 피처 가공
apps['APPS_CNT_FAM_INCOME_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['CNT_FAM_MEMBERS']

In [ ]:
# DAYS_BIRTH, DAYS_EMPLOYED 비율로 소득/자산 관련 Feature 가공
apps['APPS_EMPLOYED_BIRTH_RATIO'] = apps['DAYS_EMPLOYED']/apps['DAYS_BIRTH']
apps['APPS_INCOME_EMPLOYED_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_EMPLOYED']
apps['APPS_INCOME_BIRTH_RATIO'] = apps['AMT_INCOME_TOTAL']/apps['DAYS_BIRTH']
apps['APPS_CAR_BIRTH_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_BIRTH']
apps['APPS_CAR_EMPLOYED_RATIO'] = apps['OWN_CAR_AGE'] / apps['DAYS_EMPLOYED']

In [ ]:
print(apps.shape, prev_agg.shape)

In [ ]:
apps_all = apps.merge(prev_agg, on='SK_ID_CURR', how='left')
print(apps_all.shape)

In [ ]:
apps_all.info()

In [ ]:
object_columns = apps_all.dtypes[apps_all.dtypes == 'object'].index.tolist()
for column in object_columns:
    apps_all[column] = pd.factorize(apps_all[column])[0]

In [ ]:
apps_all.info()

# LGBMClassifier로 학습 수행

In [ ]:
# 학습 데이터 셋과 테스트 데이터 셋 분리
apps_all_train = apps_all[~apps_all['TARGET'].isnull()]
apps_all_test = apps_all[apps_all['TARGET'].isnull()]

apps_all_test = apps_all_test.drop('TARGET', axis=1)

In [ ]:
apps_all_train.columns.tolist()

In [ ]:
from sklearn.model_selection import train_test_split

ftr_app = apps_all_train.drop(['SK_ID_CURR', 'TARGET'], axis=1)
target_app = apps_all_train['TARGET']

train_x, valid_x, train_y, valid_y = train_test_split(ftr_app, target_app, test_size=0.3, random_state=2020)
train_x.shape, valid_x.shape

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf = LGBMClassifier(
        n_jobs=-1,
        n_estimators=1000,
        learning_rate=0.02,
        num_leaves=32,
        subsample=0.8,
        max_depth=12,
        silent=-1,
        verbose=-1
        )

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 100, 
        early_stopping_rounds= 100)

In [ ]:
from lightgbm import plot_importance

In [ ]:
plot_importance(clf, figsize=(16, 32))

# 테스트 데이터 결과 예측 및 Submit

In [ ]:
preds = clf.predict_proba(apps_all_test.drop('SK_ID_CURR', axis=1))[:, 1 ]

In [ ]:
apps_all_test['TARGET'] = preds
apps_all_test[['SK_ID_CURR', 'TARGET']].to_csv('prev_baseline_03.csv', index=False)